# Deposits Forecast Model with Seasonality using PyMC and Random Forest

In [ ]:
import arviz as az
import numpy as np
import pandas as pd
import pymc as pm
import plotly.express as px
import plotly.graph_objects as go

## Load data

In [ ]:
from validmind.datasets.regression import fred_deposits as demo_dataset

deposits_df, deposits_seasonality_df, fedfunds_df, tb3ms_df, gs10_df, gs30_df = demo_dataset.load_data()

df = deposits_seasonality_df.copy()

df["Month"] = df.index
df["FEDFUNDS"] = fedfunds_df["FEDFUNDS"]
df["TB3MS"] = tb3ms_df["TB3MS"]
df["GS10"] = gs10_df["GS10"]
df["GS30"] = gs30_df["GS30"]

target_column = demo_dataset.target_column

In [ ]:
fig = px.line(df, x=df["Month"], y=target_column, title='Original Data')
fig.update_layout(xaxis_title='Month', yaxis_title=target_column)
fig.show()

## Train seasonaility model

In [ ]:
t = (df["Month"]- pd.Timestamp("1900-01-01")).dt.days.to_numpy()
t_min = np.min(t)
t_max = np.max(t)
t = (t - t_min) / (t_max - t_min)

In [ ]:
y = df[target_column].to_numpy()
y_max = np.max(y)
y = y / y_max

In [ ]:
with pm.Model(check_bounds=False) as linear:
    alpha = pm.Normal("alpha", mu=0, sigma=0.5)
    beta = pm.Normal("beta", mu=0, sigma=0.5)
    sigma = pm.HalfNormal("sigma", sigma=0.5)
    trend = pm.Deterministic("trend", alpha + beta * t)
    pm.Normal("likelihood", mu=trend, sigma=sigma, observed=y)

    linear_prior = pm.sample_prior_predictive()

with linear:
    linear_trace = pm.sample(return_inferencedata=True)
    linear_prior = pm.sample_posterior_predictive(trace=linear_trace)

In [ ]:
likelihood = az.extract(linear_prior, group="posterior_predictive", num_samples=100)["likelihood"] * y_max
trend = az.extract(linear_trace, group="posterior", num_samples=100)["trend"] * y_max

In [ ]:
# Plotting the prior likelihood
fig = go.Figure()

for sample in likelihood.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.add_trace(go.Scatter(x=df['Month'], y=df[target_column], mode='lines+markers', marker=dict(color='black', size=5), name='Data'))

fig.update_layout(title="Posterior Predictive", xaxis_title="Date", yaxis_title=target_column)

fig.show()


# Plotting the trend lines
fig = go.Figure()

for sample in trend.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.add_trace(go.Scatter(x=df['Month'], y=df[target_column], mode='lines+markers', marker=dict(color='black', size=5), name='Data'))

fig.update_layout(title="Posterior Trend Lines", xaxis_title="Date", yaxis_title=target_column)

fig.show()

In [ ]:
n_order = 10
periods = (df["Month"] - pd.Timestamp("1900-01-01")).dt.days / 365.25

fourier_features = pd.DataFrame(
    {
        f"{func}_order_{order}": getattr(np, func)(2 * np.pi * periods * order)
        for order in range(1, n_order + 1)
        for func in ("sin", "cos")
    }
)
fourier_features

In [ ]:
coords = {"fourier_features": np.arange(2 * n_order)}
with pm.Model(check_bounds=False, coords=coords) as linear_with_seasonality:
    alpha = pm.Normal("alpha", mu=0, sigma=0.5)
    beta = pm.Normal("beta", mu=0, sigma=0.5)
    sigma = pm.HalfNormal("sigma", sigma=0.1)
    beta_fourier = pm.Normal("beta_fourier", mu=0, sigma=0.1, dims="fourier_features")
    seasonality = pm.Deterministic(
        "seasonality", pm.math.dot(beta_fourier, fourier_features.to_numpy().T)
    )
    trend = pm.Deterministic("trend", alpha + beta * t)
    mu = trend + seasonality
    pm.Normal("likelihood", mu=mu, sigma=sigma, observed=y)

    linear_seasonality_prior = pm.sample_prior_predictive()

In [ ]:
likelihood = az.extract(linear_seasonality_prior, group="prior_predictive", num_samples=100)["likelihood"] * y_max
trend = az.extract(linear_seasonality_prior, group="prior", num_samples=100)["trend"] * y_max
seasonality = az.extract(linear_seasonality_prior, group="prior", num_samples=100)["seasonality"] * 100

In [ ]:
# Plotting the prior likelihood
fig = go.Figure()

for sample in likelihood.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.add_trace(go.Scatter(x=df['Month'], y=df[target_column], mode='lines+markers', marker=dict(color='black', size=5), name='Data'))

fig.update_layout(title="Prior Predictive", xaxis_title="Date", yaxis_title=target_column)

fig.show()


# Plotting the trend lines
fig = go.Figure()

for sample in trend.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.add_trace(go.Scatter(x=df['Month'], y=df[target_column], mode='lines+markers', marker=dict(color='black', size=5), name='Data'))

fig.update_layout(title="Prior Trend Lines", xaxis_title="Date", yaxis_title=target_column)

fig.show()

# Plotting the seasonality lines
fig = go.Figure()

for sample in seasonality.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.update_layout(title="Prior Seasonality Lines", xaxis_title="Date", yaxis_title=target_column)

fig.show()

In [ ]:
with linear_with_seasonality:
    linear_seasonality_trace = pm.sample(return_inferencedata=True)
    linear_seasonality_posterior = pm.sample_posterior_predictive(trace=linear_seasonality_trace)

In [ ]:
likelihood = az.extract(linear_seasonality_posterior, group="posterior_predictive", num_samples=100)["likelihood"] * y_max
trend = az.extract(linear_trace, group="posterior", num_samples=100)["trend"] * y_max
seasonality = az.extract(linear_seasonality_trace, group="posterior", num_samples=100)["seasonality"] * 10000

In [ ]:
# Plotting the posterior predictive
fig = go.Figure()

for sample in likelihood.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.add_trace(go.Scatter(x=df['Month'], y=df[target_column], mode='lines+markers', marker=dict(color='black', size=5), name='Data'))

fig.update_layout(title="Posterior Predictive", xaxis_title="Date", yaxis_title=target_column)

fig.show()


# Plotting the posterior trend lines
fig = go.Figure()

for sample in trend.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.add_trace(go.Scatter(x=df['Month'], y=df[target_column], mode='lines+markers', marker=dict(color='black', size=5), name='Data'))

fig.update_layout(title="Posterior Trend Lines", xaxis_title="Date", yaxis_title=target_column)

fig.show()


# Plotting the seasonality lines
fig = go.Figure()

for sample in seasonality.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.update_layout(title="Posterior Seasonality", xaxis_title="Date", yaxis_title=target_column)

fig.show()

## Train Random Forest forecasting model

In [ ]:
# Extract the posterior predictive mean for seasonality
seasonality_posterior_mean = seasonality.mean(axis=1)

In [ ]:
# Adjust the target variable by removing the seasonality component
df[f"{target_column}_seasonal_adjusted"] = df[target_column] - seasonality_posterior_mean
df

In [ ]:
import plotly.graph_objects as go

# Create the plot
fig = go.Figure()

# Add original values to the plot
fig.add_trace(go.Scatter(x=df.index, y=df[target_column], mode='lines', name='Original'))

# Add seasonally adjusted values to the plot
fig.add_trace(go.Scatter(x=df.index, y=df[f"{target_column}_seasonal_adjusted"], mode='lines', name='Seasonally Adjusted'))

# Update the layout
fig.update_layout(
    title='Original vs Seasonally Adjusted Deposits',
    xaxis_title='Date',
    yaxis_title='Deposits',
    legend_title='Legend'
)

# Show the plot
fig.show()


In [ ]:
df

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Define the independent variables and the target variable
target_column = 'DPSACBW027NBOG_seasonal_adjusted'

independent_vars = df[['FEDFUNDS', 'TB3MS', 'GS10']]
target_var = df[target_column]

# Compute first differences for the independent variables and the target variable
independent_vars_diff = independent_vars.diff().dropna()
target_var_diff = target_var.diff().dropna()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(independent_vars_diff, target_var_diff, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_adjusted = rf_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred_adjusted)
print(f'Mean Squared Error: {mse}')

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Aligning the test indices with the original DataFrame
test_indices = X_test.index

# Create a DataFrame to hold actual and predicted values with the corresponding dates
results_df = pd.DataFrame({
    'Date': df.loc[test_indices, 'Month'],
    'Actual': y_test,
    'Predicted': y_pred_adjusted
}).set_index('Date')

# Sort the DataFrame by date to ensure the time series is ordered correctly
results_df = results_df.sort_index()


In [ ]:
results_df

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Create the plot
fig = go.Figure()

# Add actual values as a dotted line plot
fig.add_trace(go.Scatter(
    x=results_df.index, 
    y=results_df['Actual'], 
    mode='lines+markers', 
    name='Actual',
    line=dict(dash='dot', color='blue')
))

# Add predicted values as a solid line plot
fig.add_trace(go.Scatter(
    x=results_df.index, 
    y=results_df['Predicted'], 
    mode='lines+markers', 
    name='Predicted',
    line=dict(color='red')
))

# Update the layout
fig.update_layout(
    title='Predicted vs Actual Values',
    xaxis_title='Date',
    yaxis_title='Values',
    legend_title='Legend'
)

# Show the plot
fig.show()


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(results_df['Actual'], results_df['Predicted'])
print(f'Mean Squared Error (MSE): {mse}')

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse}')

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(results_df['Actual'], results_df['Predicted'])
print(f'Mean Absolute Error (MAE): {mae}')

# Calculate R-squared (R²)
r2 = r2_score(results_df['Actual'], results_df['Predicted'])
print(f'R-squared (R²): {r2}')
